In [1]:
import os

In [2]:
%pwd

'd:\\chest-cancer-classification\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\chest-cancer-classification'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class TrainingModelConfig:

    root_dir: Path
    trained_model_path: Path
    updated_model_path: Path
    training_data: Path
    params_epochs: int
    params_learning_rate: float
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list

In [6]:
from chest_cancer.constants import *
from chest_cancer.utils.commen import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_training_model_config(self) -> TrainingModelConfig:
        training = self.config.model_training
        base_model = self.config.model_selection
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "chest-cancer-ct")
        

        create_directories([
            Path(training.root_dir)
            ])

        training_model_config = TrainingModelConfig(
            root_dir = Path(training.root_dir),
            trained_model_path = Path(training.trained_model_path),
            updated_model_path = Path(base_model.updated_model_path),
            training_data = Path(training_data),
            params_epochs = params.EPOCHS,
            params_learning_rate = params.LEARNING_RATE,
            params_batch_size = params.BATCH_SIZE,
            params_is_augmentation = params.AUGMENTATION,
            params_image_size = params.IMAGE_SIZE
        )

        return training_model_config

In [8]:
import os
import time
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
tf.config.experimental_run_functions_eagerly(True)


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "d:\chest-cancer-classification\venv\lib\runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "d:\chest-cancer-classification\venv\lib\runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "d:\chest-cancer-classification\venv\lib\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "d:\chest-cancer-classification\venv\lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
 

AttributeError: _ARRAY_API not found

2025-01-22 15:56:59,089 - WARNING - deprecation - From C:\Users\ajuar\AppData\Local\Temp\ipykernel_23292\752037662.py:6: experimental_run_functions_eagerly (from tensorflow.python.eager.polymorphic_function.eager_function_run) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.config.run_functions_eagerly` instead of the experimental version.


In [9]:
class Training:
    def __init__(self, config: TrainingModelConfig):
        self.config = config

    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_model_path
        )

    def train_valid_generator(self):
        datagenerator_kwargs = dict(
            rescale=1. / 255,
            validation_split=0.20
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator

        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)

    def train(self):
        # Define a new optimizer and compile the model
        self.model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=self.config.params_learning_rate),
            loss='categorical_crossentropy',
            metrics=['accuracy']
        )

        # Compute training and validation steps
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        # Train the model
        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.valid_generator
        )

        # Save the trained model
        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )


In [10]:
try:
    config = ConfigurationManager()
    training_model_config = config.get_training_model_config()
    training = Training(config=training_model_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train()
    
except Exception as e:
    raise e

2025-01-22 15:56:59,199 - INFO - commen - yaml file: config\config.yaml loaded successfully
2025-01-22 15:56:59,205 - INFO - commen - yaml file: params.yaml loaded successfully
2025-01-22 15:56:59,207 - INFO - commen - Created directory at artifacts
2025-01-22 15:56:59,210 - INFO - commen - Created directory at artifacts\model_training
2025-01-22 15:56:59,761 - WARNING - saving_utils - Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.
Found 68 images belonging to 2 classes.
Found 275 images belonging to 2 classes.


d:\chest-cancer-classification\venv\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
d:\chest-cancer-classification\venv\lib\site-packages\tensorflow\python\data\ops\structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


8/8 ━━━━━━━━━━━━━━━━━━━━ 130s 16s/step - accuracy: 0.6112 - loss: 6.9925 - val_accuracy: 1.0000 - val_loss: 7.8294e-04
